In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from itertools import repeat


In [2]:
"""
X = dataset from user
n_destinations = number of destinations

"""

class queue_system():
    def __init__(self, X, n_destinations, n_buses, n_seats, journey_times):
        
        self.X = X
        self.n_destinations = n_destinations
        self.n_buses = n_buses
        self.n_seats = n_seats
        self.service_times = list(np.zeros(self.n_destinations))
        self.journey_times = journey_times
        self.arrival_rates = list(np.zeros(self.n_destinations))
        self.service_rates = list(np.zeros(self.n_destinations))
        self.utilization_rates = list(np.zeros(n_destinations))
        self.min_serv_time = list(np.zeros(self.n_destinations))
        self.max_serv_time = list(np.zeros(self.n_destinations))
        
        
    def check_data(self):
        '''
        Check 
        '''
        
        if self.n_destinations==len(X.columns) and self.n_destinations==len(self.n_seats) \
            and self.n_destinations==len(self.n_buses) and self.n_destinations==len(self.journey_times) \
            and self.n_destinations==len(self.min_serv_time) and n_destinations==len(self.max_serv_time):
            return True
        else:
            print("Error: Number of columns is not equal to number of destinations or ....")
            quit()
            return False
#             break -- EXIT THE PROGRAM IF FALSE
            
        
    def calculate_parameters(self):
        """
        Calculate:
        Arrival rate for each destination
        Service rate for each destination
        utilization factor for each destination
        """
#         arrival_rate = total_number_arrivals / total_time
        
        for i in range(self.n_destinations):
            min_ = min(self.X[self.X.columns[i]])
            max_ = max(self.X[self.X.columns[i]])
            total_time = max_ - min_
            total_num_arrivals = len(self.X[self.X.columns[i]])
#             self.arrival_rates[i] = total_num_arrivals/total_time
            self.min_serv_time = [x-0.5 for x in self.service_times]
            self.max_serv_time = [x+0.5 for x in self.service_times]
            
        
    
    def bus_permutation(self):
        """
        Generate random number of buses to each destination
        Randomly pick n number of seats for each destination
        """
        #1 assign random_num_buses to each queue
        self.calculate_parameters()
        total_n_buses = sum(self.n_buses)
        n_buses_in_q = []
        
        if (self.n_destinations==1):
            n_buses_in_q.append(total_n_buses)
        else:
            p = self.n_destinations - 2
            end = total_n_buses - p
            for i in range(self.n_destinations):
                n = np.random.choice(range(1, end))
                n_buses_in_q.append(n)
                end = end - n + 1
            
    
        #Create an expanded list of n_seats available
        list_repeated_n_seats = []
        for i in range(self.n_destinations):
            list_repeated_n_seats += list(repeat(self.n_seats[i],self.n_buses[i]))
            
        #for each queue that has been assigned a num_of_buses, assign n_seats
        list_total_n_seats = []
        total_seats = []
        total_n_seats_in_q = 0
        
        for i in range(self.n_destinations):
            total_n_seats_in_q = 0
            s = 0
            for j in range(n_buses_in_q[i]):
                picked_num = np.random.choice(list_repeated_n_seats)
                s += picked_num
                total_n_seats_in_q = picked_num
                list_repeated_n_seats.remove(picked_num)
                
                list_total_n_seats.append(total_n_seats_in_q)
            total_seats.append(s)
            
        return n_buses_in_q, total_seats
            
                
                
        
    def calculate_waiting_time(self):
        bus_perm, total_seats = self.bus_permutation()
        w_q_mmc = []
        w_q_ggc = []
        

        for i in range(self.n_destinations):
            sum_ = 0
            
            inter_arrival_time = self.X[self.X.columns[i]].diff()
            self.service_times[i] = self.journey_times[i]/total_seats[i]
            self.service_rates[i] = 1/self.service_times[i]            
            var_serv = (((self.max_serv_time[i] - self.min_serv_time[i])**2))/12
            var_inter = inter_arrival_time.var()
            mean_inter = inter_arrival_time.mean()
            self.arrival_rates[i] = 1/mean_inter
            self.utilization_rates[i] = self.arrival_rates[i]/(self.service_rates[i])

            c_a = var_inter / (mean_inter)**2
            c_s = var_serv/(1/self.service_rates[i])**2
            
            p = self.utilization_rates[i]
            
            l_q = ((p**2) * (1 + c_s) * (c_a + c_s*(p**2))) / (2 * (1-p) * (1 + c_s*(p**2)))
            mmc = l_q/self.arrival_rates[i]
            w_q_mmc.append(l_q/self.arrival_rates[i])
            w_q_ggc.append(mmc * ((c_a + c_s)/2))
        return bus_perm, total_seats, w_q_mmc, w_q_ggc
        
        
    def simulate(self, n_sim):
        
        dict_ = dict()
        
        for i in range(n_sim):
            bus_perm, n_seats_per_q, w_q_mmc, w_q_ggc = self.calculate_waiting_time()
            
            if ((min(w_q_mmc))>0):
                mean_waiting_time = np.mean(w_q_mmc)
                dict_[mean_waiting_time] = [w_q_mmc, bus_perm, n_seats_per_q]
        if (len(dict_) != 0):
            min_mean_waiting_time = min(dict_.keys())
            individual_wait_time, best_bus_perm, best_n_seats_per_q = dict_[min_mean_waiting_time]
        else:
            print("All reallocations produced an unstable system")
            return None
        
        return min_mean_waiting_time, individual_wait_time, best_bus_perm, best_n_seats_per_q

In [4]:
Town = pd.read_excel("Town.xlsx", sheet_name='Passenger Arrival Time')
Kabuga = pd.read_excel('Kabuga.xls', sheet_name='Passenger Arrival Time')
Zindiro = pd.read_excel('Zindiro.xlsx', sheet_name='Passenger Arrival Time')

X_combined = Zindiro[['arrival in minute']]
X_combined = X_combined.rename(columns={"arrival in minute" : "Zindiro"})
# rename(columns={"A": "a", "B": "c"})
X_combined['Town'] = Town['Arrival Time in Minutes']
X_combined['Kabuga'] = Kabuga['Arrival in Minutes']
X_combined_Kabuga = X_combined[['Town', 'Kabuga']]
X_combined_Zindiro = X_combined[['Town', 'Zindiro']]


In [8]:
# Reallocation of buses for Zindiro, Town and Kabuga
n_seats_all = [83, 22, 22]
journey_times_all = [60, 75, 55]
n_buses_all = [4, 10, 4]
n_destinations_all = 3

q_all = queue_system(X_combined, n_destinations_all, n_buses_all, n_seats_all, journey_times_all)
sim_q_all = q_all.simulate(10000)
print(sim_q_all)
print('The mean waiting time: ' + str(sim_q_all[0]))
print('The average waiting time of Town is: ' + str(sim_q_all[1][1]))
print('The number of buses assigned to Town is: ' + str(sim_q_all[2][1]))
print('The number of seats assigned to Town is: ' + str(sim_q_all[3][1]))
print('The average waiting time of Zindiro is: ' + str(sim_q_all[1][0]))
print('The number of buses assigned to Zindiro is: ' + str(sim_q_all[2][0]))
print('The number of seats assigned to Zindiro is: ' + str(sim_q_all[3][0]))
print('The average waiting time of Kabuga is: ' + str(sim_q_all[1][2]))
print('The number of buses assigned to Kabuga is: ' + str(sim_q_all[2][2]))
print('The number of seats assigned to Kabuga is: ' + str(sim_q_all[3][2]))


(5.379313228080053, [6.51143446155667, 3.9472769444647837, 5.679228278218704], [10, 6, 2], [342, 193, 105])
The mean waiting time: 5.379313228080053
The average waiting time of Town is: 3.9472769444647837
The number of buses assigned to Town is: 6
The number of seats assigned to Town is: 193
The average waiting time of Zindiro is: 6.51143446155667
The number of buses assigned to Zindiro is: 10
The number of seats assigned to Zindiro is: 342
The average waiting time of Kabuga is: 5.679228278218704
The number of buses assigned to Kabuga is: 2
The number of seats assigned to Kabuga is: 105


In [6]:
# Reallocation of buses for Town and Zindiro
n_seats_z = [22, 83]
journey_times_z = [75, 60]
n_buses_z = [10, 4]
n_destinations_z = 2

q_z = queue_system(X_combined_Zindiro, n_destinations_z, n_buses_z, n_seats_z, journey_times_z)
sim_q_z = q_z.simulate(10000)
print(sim_q_z)
print('The mean waiting time: ' + str(sim_q_z[0]))
print('The average waiting time of Town is: ' + str(sim_q_z[1][0]))
print('The number of buses assigned to Town is: ' + str(sim_q_z[2][0]))
print('The number of seats assigned to Town is: ' + str(sim_q_z[3][0]))
print('The average waiting time of Zindiro is: ' + str(sim_q_z[1][1]))
print('The number of buses assigned to Zindiro is: ' + str(sim_q_z[2][1]))
print('The number of seats assigned to Zindiro is: ' + str(sim_q_z[3][1]))

(4.370822433421307, [3.9472769444647837, 4.79436792237783], [6, 8], [193, 359])
The mean waiting time: 4.370822433421307
The average waiting time of Town is: 3.9472769444647837
The number of buses assigned to Town is: 6
The number of seats assigned to Town is: 193
The average waiting time of Zindiro is: 4.79436792237783
The number of buses assigned to Zindiro is: 8
The number of seats assigned to Zindiro is: 359


In [7]:
# Reallocation of buses for Town and Kabuga
n_seats_k = [22, 22]
journey_times_k = [75, 55]
n_buses_k = [10, 4]
n_destinations_k = 2

q_k = queue_system(X_combined_Kabuga, n_destinations_k, n_buses_k, n_seats_k, journey_times_k)
sim_q_k = q_k.simulate(10000)
print(sim_q_k)
print('The mean waiting time: ' + str(sim_q_k[0]))
print('The average waiting time of Town is: ' + str(sim_q_k[1][0]))
print('The number of buses assigned to Town is: ' + str(sim_q_k[2][0]))
print('The number of seats assigned to Town is: ' + str(sim_q_k[3][0]))
print('The average waiting time of Kabuga is: ' + str(sim_q_k[1][1]))
print('The number of buses assigned to Kabuga is: ' + str(sim_q_k[2][1]))
print('The number of seats assigned to Kabuga is: ' + str(sim_q_k[3][1]))

(3.659758642630754, [3.45536196442385, 3.8641553208376576], [9, 5], [198, 110])
The mean waiting time: 3.659758642630754
The average waiting time of Town is: 3.45536196442385
The number of buses assigned to Town is: 9
The number of seats assigned to Town is: 198
The average waiting time of Kabuga is: 3.8641553208376576
The number of buses assigned to Kabuga is: 5
The number of seats assigned to Kabuga is: 110
